In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
#Load the dataset
true=pd.read_csv("true.csv")

In [3]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
#Load the dataset
fake=pd.read_csv("fake.csv")

In [5]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
fake["class"] = 0  # 0 for fake news
true["class"] = 1  # 1 for real news

In [7]:
fake.shape, true.shape

((23481, 5), (21417, 5))

In [8]:
data_fake_manual_testing = fake.tail(10)
for i in range(23480,23470, -1):
    fake.drop([i], axis = 0, inplace = True)

data_true_manual_testing = true.tail(10)
for i in range(21416,21406, -1):
    true.drop([i], axis = 0, inplace = True)

In [9]:
fake.shape, true.shape

((23471, 5), (21407, 5))

In [10]:
data_fake_manual_testing["class"] = 0  # 0 for fake news
data_true_manual_testing["class"] = 1  # 1 for real news

C:\Users\ASUS\AppData\Local\Temp\ipykernel_3744\4051607853.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing["class"] = 0  # 0 for fake news
C:\Users\ASUS\AppData\Local\Temp\ipykernel_3744\4051607853.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing["class"] = 1  # 1 for real news


In [11]:
data_fake_manual_testing.head()

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0


In [12]:
data_true_manual_testing.head()

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1


In [13]:
# Combine datasets
data_merge=pd.concat([fake,true], axis = 0)
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [14]:
print(data_merge.columns)

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')


In [15]:
data = data_merge.drop(["title", "subject", "date"], axis=1)

In [16]:
# Shuffle dataset
data=data.sample(frac=1)

In [17]:
# Keep only the review_description column
data1 = data[['text']]

# Display the updated DataFrame
print(data1)

                                                    text
8110   NEW YORK (Reuters) - An election analysis cond...
15474  EDINBURGH (Reuters) - Three similar envelopes,...
15291  Will this news only increase Obama s desire to...
15408  MANILA (Reuters) - Philippine President Rodrig...
17696  HONG KONG/BEIJING (Reuters) - China said on Th...
...                                                  ...
20906  No woman should be working in these public hou...
20133  Will Hillary s health be the only thing she ca...
23416   21st Century Wire asks HAVE YOUR SHOUT: Frida...
4797   Following the mass exodus of Hispanic surrogat...
538    While the President of the United States seems...

[44878 rows x 1 columns]


In [24]:
data1.head()

,text
8110,NEW YORK (Reuters) - An election analysis cond...
15474,"EDINBURGH (Reuters) - Three similar envelopes,..."
15291,Will this news only increase Obama s desire to...
15408,MANILA (Reuters) - Philippine President Rodrig...
17696,HONG KONG/BEIJING (Reuters) - China said on Th...


In [25]:
data1.reset_index(inplace=True)
data1.drop(["index"], axis=1, inplace=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_3744\1941005049.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.drop(["index"], axis=1, inplace=True)


In [26]:
data1.head()

,text
0,NEW YORK (Reuters) - An election analysis cond...
1,"EDINBURGH (Reuters) - Three similar envelopes,..."
2,Will this news only increase Obama s desire to...
3,MANILA (Reuters) - Philippine President Rodrig...
4,HONG KONG/BEIJING (Reuters) - China said on Th...


In [27]:
data1.isnull().sum()

text    0
dtype: int64

In [28]:
data1.duplicated().sum()

6241

In [29]:
df=data1.drop_duplicates(inplace=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_3744\1384147895.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df=data1.drop_duplicates(inplace=True)


In [30]:
data1.duplicated().sum()

0

In [31]:
import nltk

# Download the popular datasets and corpora
nltk.download('popular')  # This downloads the most commonly used NLTK datasets

# Download the part-of-speech tagger
nltk.download('averaged_perceptron_tagger')  # This downloads the POS tagger

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |   

True

In [32]:
# Cleaning the text
def clean(text):
    text = re.sub('[^A-Za-z]+', ' ', text)
    return text

data1['Cleaned text'] = data1['text'].apply(clean)
data1.head()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_3744\210048941.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['Cleaned text'] = data1['text'].apply(clean)


,text,Cleaned text
0,NEW YORK (Reuters) - An election analysis cond...,NEW YORK Reuters An election analysis conducte...
1,"EDINBURGH (Reuters) - Three similar envelopes,...",EDINBURGH Reuters Three similar envelopes one ...
2,Will this news only increase Obama s desire to...,Will this news only increase Obama s desire to...
3,MANILA (Reuters) - Philippine President Rodrig...,MANILA Reuters Philippine President Rodrigo Du...
4,HONG KONG/BEIJING (Reuters) - China said on Th...,HONG KONG BEIJING Reuters China said on Thursd...


In [33]:
data1['Lowercase'] = data1['Cleaned text'].apply(lambda x: " ".join(word.lower() for word in x.split()))

C:\Users\ASUS\AppData\Local\Temp\ipykernel_3744\293980780.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['Lowercase'] = data1['Cleaned text'].apply(lambda x: " ".join(word.lower() for word in x.split()))


In [34]:
data1.head(10)

,text,Cleaned text,Lowercase
0,NEW YORK (Reuters) - An election analysis cond...,NEW YORK Reuters An election analysis conducte...,new york reuters an election analysis conducte...
1,"EDINBURGH (Reuters) - Three similar envelopes,...",EDINBURGH Reuters Three similar envelopes one ...,edinburgh reuters three similar envelopes one ...
2,Will this news only increase Obama s desire to...,Will this news only increase Obama s desire to...,will this news only increase obama s desire to...
3,MANILA (Reuters) - Philippine President Rodrig...,MANILA Reuters Philippine President Rodrigo Du...,manila reuters philippine president rodrigo du...
4,HONG KONG/BEIJING (Reuters) - China said on Th...,HONG KONG BEIJING Reuters China said on Thursd...,hong kong beijing reuters china said on thursd...
5,It has been sixteen years since the American e...,It has been sixteen years since the American e...,it has been sixteen years since the american e...
6,It appears that Trump tried everything he coul...,It appears that Trump tried everything he coul...,it appears that trump tried everything he coul...
7,Republicans in the state of North Carolina rea...,Republicans in the state of North Carolina rea...,republicans in the state of north carolina rea...
8,MOSCOW (Reuters) - Russian President Vladimir ...,MOSCOW Reuters Russian President Vladimir Puti...,moscow reuters russian president vladimir puti...
9,I m convinced the Freedom From Religion group...,I m convinced the Freedom From Religion group...,i m convinced the freedom from religion group ...


In [35]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [36]:
data1['stopwords'] = data1['Lowercase'].apply(lambda x: " ".join(word for word in x.split() if word not in stop_words))

C:\Users\ASUS\AppData\Local\Temp\ipykernel_3744\3101138197.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['stopwords'] = data1['Lowercase'].apply(lambda x: " ".join(word for word in x.split() if word not in stop_words))


In [37]:
data1.head(10)

,text,Cleaned text,Lowercase,stopwords
0,NEW YORK (Reuters) - An election analysis cond...,NEW YORK Reuters An election analysis conducte...,new york reuters an election analysis conducte...,new york reuters election analysis conducted r...
1,"EDINBURGH (Reuters) - Three similar envelopes,...",EDINBURGH Reuters Three similar envelopes one ...,edinburgh reuters three similar envelopes one ...,edinburgh reuters three similar envelopes one ...
2,Will this news only increase Obama s desire to...,Will this news only increase Obama s desire to...,will this news only increase obama s desire to...,news increase obama desire work harder stoke r...
3,MANILA (Reuters) - Philippine President Rodrig...,MANILA Reuters Philippine President Rodrigo Du...,manila reuters philippine president rodrigo du...,manila reuters philippine president rodrigo du...
4,HONG KONG/BEIJING (Reuters) - China said on Th...,HONG KONG BEIJING Reuters China said on Thursd...,hong kong beijing reuters china said on thursd...,hong kong beijing reuters china said thursday ...
5,It has been sixteen years since the American e...,It has been sixteen years since the American e...,it has been sixteen years since the american e...,sixteen years since american economy added man...
6,It appears that Trump tried everything he coul...,It appears that Trump tried everything he coul...,it appears that trump tried everything he coul...,appears trump tried everything could think dis...
7,Republicans in the state of North Carolina rea...,Republicans in the state of North Carolina rea...,republicans in the state of north carolina rea...,republicans state north carolina really need g...
8,MOSCOW (Reuters) - Russian President Vladimir ...,MOSCOW Reuters Russian President Vladimir Puti...,moscow reuters russian president vladimir puti...,moscow reuters russian president vladimir puti...
9,I m convinced the Freedom From Religion group...,I m convinced the Freedom From Religion group...,i m convinced the freedom from religion group ...,convinced freedom religion group atheists get ...


In [38]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk import pos_tag

# Download necessary NLTK data files
nltk.download('punkt_tab')
nltk.download('wordnet')

# POS tagger dictionary
pos_dict = {'J': wordnet.ADJ, 'V': wordnet.VERB, 'N': wordnet.NOUN, 'R': wordnet.ADV}

def token_pos(text):
    return [
        (word, pos_dict.get(tag[0]))
        for word, tag in pos_tag(word_tokenize(text))
    ]

# Apply to your data
data1['POS tagged'] = data1['stopwords'].apply(token_pos)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\ASUS\AppData\Local\Temp\ipykernel_3744\138016589.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['POS tagged'] = data1['stopwords'].apply(token_pos)


In [39]:
data1.head()

,text,Cleaned text,Lowercase,stopwords,POS tagged
0,NEW YORK (Reuters) - An election analysis cond...,NEW YORK Reuters An election analysis conducte...,new york reuters an election analysis conducte...,new york reuters election analysis conducted r...,"[(new, a), (york, n), (reuters, n), (election,..."
1,"EDINBURGH (Reuters) - Three similar envelopes,...",EDINBURGH Reuters Three similar envelopes one ...,edinburgh reuters three similar envelopes one ...,edinburgh reuters three similar envelopes one ...,"[(edinburgh, n), (reuters, n), (three, None), ..."
2,Will this news only increase Obama s desire to...,Will this news only increase Obama s desire to...,will this news only increase obama s desire to...,news increase obama desire work harder stoke r...,"[(news, n), (increase, n), (obama, n), (desire..."
3,MANILA (Reuters) - Philippine President Rodrig...,MANILA Reuters Philippine President Rodrigo Du...,manila reuters philippine president rodrigo du...,manila reuters philippine president rodrigo du...,"[(manila, n), (reuters, n), (philippine, v), (..."
4,HONG KONG/BEIJING (Reuters) - China said on Th...,HONG KONG BEIJING Reuters China said on Thursd...,hong kong beijing reuters china said on thursd...,hong kong beijing reuters china said thursday ...,"[(hong, r), (kong, r), (beijing, v), (reuters,..."


In [40]:
from textblob import Word
from nltk.stem import WordNetLemmatizer

In [41]:
# Initialize the lemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
        if not pos:  # If no POS tag is available, use the word as is
            lemma = word
        else:
            # Lemmatize based on the POS tag
            lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
        lemma_rew = lemma_rew + " " + lemma  # Add the lemma to the result string
    return lemma_rew

# Assuming df2 is your dataframe and 'POS tagged' contains POS-tagged text
data1['Lemma'] = data1['POS tagged'].apply(lemmatize)

# Display the first few rows of the dataframe
data1.head()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_3744\926378792.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['Lemma'] = data1['POS tagged'].apply(lemmatize)


,text,Cleaned text,Lowercase,stopwords,POS tagged,Lemma
0,NEW YORK (Reuters) - An election analysis cond...,NEW YORK Reuters An election analysis conducte...,new york reuters an election analysis conducte...,new york reuters election analysis conducted r...,"[(new, a), (york, n), (reuters, n), (election,...",new york reuters election analysis conduct r...
1,"EDINBURGH (Reuters) - Three similar envelopes,...",EDINBURGH Reuters Three similar envelopes one ...,edinburgh reuters three similar envelopes one ...,edinburgh reuters three similar envelopes one ...,"[(edinburgh, n), (reuters, n), (three, None), ...",edinburgh reuters three similar envelope one...
2,Will this news only increase Obama s desire to...,Will this news only increase Obama s desire to...,will this news only increase obama s desire to...,news increase obama desire work harder stoke r...,"[(news, n), (increase, n), (obama, n), (desire...",news increase obama desire work harder stoke...
3,MANILA (Reuters) - Philippine President Rodrig...,MANILA Reuters Philippine President Rodrigo Du...,manila reuters philippine president rodrigo du...,manila reuters philippine president rodrigo du...,"[(manila, n), (reuters, n), (philippine, v), (...",manila reuters philippine president rodrigo ...
4,HONG KONG/BEIJING (Reuters) - China said on Th...,HONG KONG BEIJING Reuters China said on Thursd...,hong kong beijing reuters china said on thursd...,hong kong beijing reuters china said thursday ...,"[(hong, r), (kong, r), (beijing, v), (reuters,...",hong kong beijing reuters china say thursday...


In [42]:
data1['text'].iloc[0]

'NEW YORK (Reuters) - An election analysis conducted in the Reuters/Ipsos States of the Nation project shows that the race has tightened considerably over the past few weeks, with Republican presidential nominee Donald Trump projected to win Florida, an essential battleground state, if the election were held today.   The project, which is based on a weekly tracking poll of more than 15,000 Americans, shows that the 2016 presidential race could end in a photo finish on Nov. 8, with the major-party candidates running nearly even in the Electoral College, the body that ultimately selects the president.   The States of the Nation project, which delivers a weekly tally of support for the candidates in every state, shows that the race has tightened in several traditional battlegrounds. Pennsylvania has been moved from a likely win for Democratic presidential nominee Hillary Clinton to a tossup; Ohio has been moved from a tossup to a likely win for Clinton. And Florida is now considered a lik

In [43]:
data1['Lemma'].iloc[0]

'  new york reuters election analysis conduct reuters ipsos state nation project show race tighten considerably past week republican presidential nominee donald trump project win florida essential battleground state election hold today project base weekly track poll american show presidential race could end photo finish nov major party candidate run nearly even electoral college body ultimately select president state nation project deliver weekly tally support candidate every state show race tighten several traditional battleground pennsylvania move likely win democratic presidential nominee hillary clinton tossup ohio move tossup likely win clinton florida consider likely win republican nominee percent support trump percent support clinton election hold today project estimate clinton percent chance win electoral vote last week project estimate clinton percent chance win election separate national reuters ipsos track poll clinton continue lead trump percentage point recent bout pneumon

In [44]:
# Using TEXTBLOB to analyze the sentiment of the text by returning a polarity and subjectivity score.
import pandas as pd
from textblob import TextBlob

def get_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.1:  # Positive
        return 'Positive'
    elif polarity < -0.1:  # Negative
        return 'Negative'
    else:  # Neutral
        return 'Neutral'

# Apply the function to the 'Lemma' column (or 'Cleaned Reviews')
data1['sentiment'] = data1['Lemma'].apply(get_sentiment)

# Display the dataframe to check the sentiment column
print(data1.head())

                                                text  \
0  NEW YORK (Reuters) - An election analysis cond...   
1  EDINBURGH (Reuters) - Three similar envelopes,...   
2  Will this news only increase Obama s desire to...   
3  MANILA (Reuters) - Philippine President Rodrig...   
4  HONG KONG/BEIJING (Reuters) - China said on Th...   

                                        Cleaned text  \
0  NEW YORK Reuters An election analysis conducte...   
1  EDINBURGH Reuters Three similar envelopes one ...   
2  Will this news only increase Obama s desire to...   
3  MANILA Reuters Philippine President Rodrigo Du...   
4  HONG KONG BEIJING Reuters China said on Thursd...   

                                           Lowercase  \
0  new york reuters an election analysis conducte...   
1  edinburgh reuters three similar envelopes one ...   
2  will this news only increase obama s desire to...   
3  manila reuters philippine president rodrigo du...   
4  hong kong beijing reuters china said on thu

C:\Users\ASUS\AppData\Local\Temp\ipykernel_3744\853568704.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['sentiment'] = data1['Lemma'].apply(get_sentiment)


In [45]:
data1

,text,Cleaned text,Lowercase,stopwords,POS tagged,Lemma,sentiment
0,NEW YORK (Reuters) - An election analysis cond...,NEW YORK Reuters An election analysis conducte...,new york reuters an election analysis conducte...,new york reuters election analysis conducted r...,"[(new, a), (york, n), (reuters, n), (election,...",new york reuters election analysis conduct r...,Neutral
1,"EDINBURGH (Reuters) - Three similar envelopes,...",EDINBURGH Reuters Three similar envelopes one ...,edinburgh reuters three similar envelopes one ...,edinburgh reuters three similar envelopes one ...,"[(edinburgh, n), (reuters, n), (three, None), ...",edinburgh reuters three similar envelope one...,Neutral
2,Will this news only increase Obama s desire to...,Will this news only increase Obama s desire to...,will this news only increase obama s desire to...,news increase obama desire work harder stoke r...,"[(news, n), (increase, n), (obama, n), (desire...",news increase obama desire work harder stoke...,Negative
3,MANILA (Reuters) - Philippine President Rodrig...,MANILA Reuters Philippine President Rodrigo Du...,manila reuters philippine president rodrigo du...,manila reuters philippine president rodrigo du...,"[(manila, n), (reuters, n), (philippine, v), (...",manila reuters philippine president rodrigo ...,Positive
4,HONG KONG/BEIJING (Reuters) - China said on Th...,HONG KONG BEIJING Reuters China said on Thursd...,hong kong beijing reuters china said on thursd...,hong kong beijing reuters china said thursday ...,"[(hong, r), (kong, r), (beijing, v), (reuters,...",hong kong beijing reuters china say thursday...,Neutral
...,...,...,...,...,...,...,...
44871,MADRID (Reuters) - Spain s upper house Senate ...,MADRID Reuters Spain s upper house Senate coul...,madrid reuters spain s upper house senate coul...,madrid reuters spain upper house senate could ...,"[(madrid, a), (reuters, n), (spain, v), (upper...",madrid reuters spain upper house senate coul...,Neutral
44872,Abigail Disney is an heiress with brass ovarie...,Abigail Disney is an heiress with brass ovarie...,abigail disney is an heiress with brass ovarie...,abigail disney heiress brass ovaries profit go...,"[(abigail, n), (disney, n), (heiress, n), (bra...",abigail disney heiress brass ovary profit go...,Positive
44873,No woman should be working in these public hou...,No woman should be working in these public hou...,no woman should be working in these public hou...,woman working public housing units surrouded m...,"[(woman, n), (working, v), (public, a), (housi...",woman work public housing unit surrouded men...,Neutral
44876,Following the mass exodus of Hispanic surrogat...,Following the mass exodus of Hispanic surrogat...,following the mass exodus of hispanic surrogat...,following mass exodus hispanic surrogates trum...,"[(following, v), (mass, n), (exodus, n), (hisp...",follow mass exodus hispanic surrogate trump ...,Neutral


In [46]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

In [47]:
# Map sentiment to numerical values
sentiment_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
data1['sentiment_label'] = data1['sentiment'].map(sentiment_mapping)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_3744\2249151546.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['sentiment_label'] = data1['sentiment'].map(sentiment_mapping)


In [48]:
# Define features (X) and target (y)
X = data1['Lemma']  # or df['Cleaned Reviews']
y = data1['sentiment_label']

In [49]:
vectorizer = CountVectorizer()
X_vectorized = vectorizer.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

In [50]:
# Train Naive Bayes model
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [51]:
# Predictions
y_pred = model.predict(X_test)

In [52]:
# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6579968944099379
Classification Report:
               precision    recall  f1-score   support

           0       0.27      0.10      0.14       386
           1       0.79      0.72      0.75      5426
           2       0.43      0.59      0.50      1916

    accuracy                           0.66      7728
   macro avg       0.50      0.47      0.46      7728
weighted avg       0.67      0.66      0.66      7728



In [53]:
# Example prediction
new_reviews = ["Amazing app with great features!", "Not worth the time."]
new_reviews_vectorized = vectorizer.transform(new_reviews)
predictions = model.predict(new_reviews_vectorized)
print("Predictions for new reviews:", predictions)

Predictions for new reviews: [2 1]


In [54]:
data1['sentiment_label'].value_counts()

sentiment_label
1    27155
2     9510
0     1972
Name: count, dtype: int64

In [55]:
data1

,text,Cleaned text,Lowercase,stopwords,POS tagged,Lemma,sentiment,sentiment_label
0,NEW YORK (Reuters) - An election analysis cond...,NEW YORK Reuters An election analysis conducte...,new york reuters an election analysis conducte...,new york reuters election analysis conducted r...,"[(new, a), (york, n), (reuters, n), (election,...",new york reuters election analysis conduct r...,Neutral,1
1,"EDINBURGH (Reuters) - Three similar envelopes,...",EDINBURGH Reuters Three similar envelopes one ...,edinburgh reuters three similar envelopes one ...,edinburgh reuters three similar envelopes one ...,"[(edinburgh, n), (reuters, n), (three, None), ...",edinburgh reuters three similar envelope one...,Neutral,1
2,Will this news only increase Obama s desire to...,Will this news only increase Obama s desire to...,will this news only increase obama s desire to...,news increase obama desire work harder stoke r...,"[(news, n), (increase, n), (obama, n), (desire...",news increase obama desire work harder stoke...,Negative,0
3,MANILA (Reuters) - Philippine President Rodrig...,MANILA Reuters Philippine President Rodrigo Du...,manila reuters philippine president rodrigo du...,manila reuters philippine president rodrigo du...,"[(manila, n), (reuters, n), (philippine, v), (...",manila reuters philippine president rodrigo ...,Positive,2
4,HONG KONG/BEIJING (Reuters) - China said on Th...,HONG KONG BEIJING Reuters China said on Thursd...,hong kong beijing reuters china said on thursd...,hong kong beijing reuters china said thursday ...,"[(hong, r), (kong, r), (beijing, v), (reuters,...",hong kong beijing reuters china say thursday...,Neutral,1
...,...,...,...,...,...,...,...,...
44871,MADRID (Reuters) - Spain s upper house Senate ...,MADRID Reuters Spain s upper house Senate coul...,madrid reuters spain s upper house senate coul...,madrid reuters spain upper house senate could ...,"[(madrid, a), (reuters, n), (spain, v), (upper...",madrid reuters spain upper house senate coul...,Neutral,1
44872,Abigail Disney is an heiress with brass ovarie...,Abigail Disney is an heiress with brass ovarie...,abigail disney is an heiress with brass ovarie...,abigail disney heiress brass ovaries profit go...,"[(abigail, n), (disney, n), (heiress, n), (bra...",abigail disney heiress brass ovary profit go...,Positive,2
44873,No woman should be working in these public hou...,No woman should be working in these public hou...,no woman should be working in these public hou...,woman working public housing units surrouded m...,"[(woman, n), (working, v), (public, a), (housi...",woman work public housing unit surrouded men...,Neutral,1
44876,Following the mass exodus of Hispanic surrogat...,Following the mass exodus of Hispanic surrogat...,following the mass exodus of hispanic surrogat...,following mass exodus hispanic surrogates trum...,"[(following, v), (mass, n), (exodus, n), (hisp...",follow mass exodus hispanic surrogate trump ...,Neutral,1


In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(data1['Lemma'])

In [57]:
model = MultinomialNB(alpha=0.5)  # Experiment with values like 0.1, 0.5, or 1.0

In [58]:
def get_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.2:  # Positive threshold
        return 'Positive'
    elif polarity < -0.2:  # Negative threshold
        return 'Negative'
    else:
        return 'Neutral'

In [59]:
data1.head()

,text,Cleaned text,Lowercase,stopwords,POS tagged,Lemma,sentiment,sentiment_label
0,NEW YORK (Reuters) - An election analysis cond...,NEW YORK Reuters An election analysis conducte...,new york reuters an election analysis conducte...,new york reuters election analysis conducted r...,"[(new, a), (york, n), (reuters, n), (election,...",new york reuters election analysis conduct r...,Neutral,1
1,"EDINBURGH (Reuters) - Three similar envelopes,...",EDINBURGH Reuters Three similar envelopes one ...,edinburgh reuters three similar envelopes one ...,edinburgh reuters three similar envelopes one ...,"[(edinburgh, n), (reuters, n), (three, None), ...",edinburgh reuters three similar envelope one...,Neutral,1
2,Will this news only increase Obama s desire to...,Will this news only increase Obama s desire to...,will this news only increase obama s desire to...,news increase obama desire work harder stoke r...,"[(news, n), (increase, n), (obama, n), (desire...",news increase obama desire work harder stoke...,Negative,0
3,MANILA (Reuters) - Philippine President Rodrig...,MANILA Reuters Philippine President Rodrigo Du...,manila reuters philippine president rodrigo du...,manila reuters philippine president rodrigo du...,"[(manila, n), (reuters, n), (philippine, v), (...",manila reuters philippine president rodrigo ...,Positive,2
4,HONG KONG/BEIJING (Reuters) - China said on Th...,HONG KONG BEIJING Reuters China said on Thursd...,hong kong beijing reuters china said on thursd...,hong kong beijing reuters china said thursday ...,"[(hong, r), (kong, r), (beijing, v), (reuters,...",hong kong beijing reuters china say thursday...,Neutral,1


In [60]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from textblob import TextBlob

In [61]:
# Step 1: Create a sentiment column using TextBlob
def get_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.2:  # Positive threshold
        return 'Positive'
    elif polarity < -0.2:  # Negative threshold
        return 'Negative'
    else:
        return 'Neutral'

In [62]:

data1['sentiment'] = data1['Lemma'].apply(get_sentiment)  

C:\Users\ASUS\AppData\Local\Temp\ipykernel_3744\2302681549.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['sentiment'] = data1['Lemma'].apply(get_sentiment)


In [63]:
# Step 2: Map sentiments to numerical labels
sentiment_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
data1['sentiment_label'] = data1['sentiment'].map(sentiment_mapping)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_3744\3418893397.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['sentiment_label'] = data1['sentiment'].map(sentiment_mapping)


In [64]:
X = data1['Lemma']
y = data1['sentiment_label']

In [65]:
# Step 4: Vectorize text using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # Limit features to 5000 for efficiency
X_vectorized = vectorizer.fit_transform(X)

In [66]:
# Step 5: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

In [67]:
# Step 6: Train a Logistic Regression model
model = LogisticRegression(class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', random_state=42)

In [68]:
# Step 7: Make predictions and evaluate the model
y_pred = model.predict(X_test)

In [88]:
# Print evaluation metrics
print("Accuracy is:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(
    y_test, 
    y_pred, 
    target_names=['Negative', 'Neutral', 'Positive'], 
    labels=[0, 1, 2]  # Ensure it matches your sentiment_mapping
))

Accuracy is: 0.8869047619047619
Classification Report:
               precision    recall  f1-score   support

    Negative       0.14      0.45      0.21        75
     Neutral       0.98      0.90      0.94      7178
    Positive       0.41      0.82      0.55       475

    accuracy                           0.89      7728
   macro avg       0.51      0.72      0.57      7728
weighted avg       0.94      0.89      0.91      7728



In [70]:
new_reviews = ["Great app, very useful!", "Terrible experience with customer support.", "It was okay, nothing special."]
new_reviews_vectorized = vectorizer.transform(new_reviews)
predictions = model.predict(new_reviews_vectorized)

In [71]:
# Map predictions back to sentiment labels
predicted_labels = [list(sentiment_mapping.keys())[list(sentiment_mapping.values()).index(label)] for label in predictions]
print("Predictions for new reviews:", predicted_labels)

Predictions for new reviews: ['Positive', 'Negative', 'Positive']


In [83]:
model = LogisticRegression(max_iter=100, C=0.05, penalty='l2', solver='liblinear')
model.fit(X_train, y_train)

LogisticRegression(C=0.05, solver='liblinear')

In [84]:
pred_lr = model.predict(X_test)

In [87]:
# Print evaluation metrics
print("Accuracy is:", accuracy_score(y_test,pred_lr))
print("Classification Report:\n", classification_report(y_test,pred_lr, target_names=['Negative', 'Neutral', 'Positive'], 
    labels=[0, 1, 2]  # Ensure it matches your sentiment_mapping
))

Accuracy is: 0.9288302277432712
Classification Report:
               precision    recall  f1-score   support

    Negative       0.00      0.00      0.00        75
     Neutral       0.93      1.00      0.96      7178
    Positive       0.00      0.00      0.00       475

    accuracy                           0.93      7728
   macro avg       0.31      0.33      0.32      7728
weighted avg       0.86      0.93      0.89      7728



C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [89]:
DT = DecisionTreeClassifier(max_depth=1, min_samples_split=3, min_samples_leaf=2, random_state=42)
DT.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=1, min_samples_leaf=2, min_samples_split=3,
                       random_state=42)

In [90]:
pred_dt = DT.predict(X_test)

In [91]:
# Print evaluation metrics
print("Accuracy is:", accuracy_score(y_test,pred_dt))
print("Classification Report:\n", classification_report(y_test,pred_dt, target_names=['Negative', 'Neutral', 'Positive'], 
    labels=[0, 1, 2]  # Ensure it matches your sentiment_mapping
))

Accuracy is: 0.9298654244306418
Classification Report:
               precision    recall  f1-score   support

    Negative       0.00      0.00      0.00        75
     Neutral       0.93      0.99      0.96      7178
    Positive       0.54      0.11      0.18       475

    accuracy                           0.93      7728
   macro avg       0.49      0.37      0.38      7728
weighted avg       0.90      0.93      0.91      7728



C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
